# Convert SG Models to CoreML

Let's install the minimal dependencies for getting started.<br>
Those will be 'super-gradients' and 'coremltools'.<br>
If you are using Colab, please restart the kernel after they are installed and only then continue.

In [1]:
!python3 -m pip install super-gradients coremltools onnx==1.13.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Restart the notebook, and then continue;

Let's import everything we need, including super gradients and coremltools.

In [1]:
from typing import List
import os

import coremltools as ct
from super_gradients.common.object_names import Models
from super_gradients.training import models
import torch

[2023-05-15 10:56:48] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


The console stream is logged into /root/sg_logs/console.log


/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
[2023-05-15 10:57:00] WARNING - __init__.py - Failed to import pytorch_quantization
[2023-05-15 10:57:00] WARNING - calibrator.py - Failed to import pytorch_quantization
[2023-05-15 10:57:00] WARNING - export.py - Failed to import pytorch_quantization
[2023-05-15 10:57:00] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization


## Load any SG model

For this tutorial we will use the YOLO-NAS-S model.<br>
We will load the pretrained version of Deci's YOLO NAS S model and convert the graph representation from PyTorch to CoreML.<br>
We also must define an example input - It is required when exporting torch models.<br>

In [2]:
sg_torch_model = models.get(Models.YOLO_NAS_S, pretrained_weights="coco")
example_inputs = [torch.rand(1,3,640,640).cpu()]
sg_torch_model.prep_model_for_conversion(input_size=example_inputs[0].shape)
print('Loaded successfully')

[2023-05-15 10:57:00] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.


Let's define the conversion method, that receives an SG nn.Module together with eample inputs and target path, converts the model representation to CoreML, and saves the compiled checkpoint to the specified path.

In [3]:
def convert_sg_model_to_coreml(sg_model: torch.nn.Module,
                               example_inputs: List[torch.nn.Module],
                               output_path: os.PathLike = None,
                               export_as_ml_program=False):
    """
    Converts a given SG model to CoreML mlprogram or package.

    @param sg_model: A super-gradients nn.Module to compile
    @param example_inputs: A list of input tensors to feed the model, required to trace with torch.jit.
    @param output_path: The path to save the compiled CoreML model into.
    @param export_as_ml_program: Whether to convert to the new program format (better) or legacy coreml proto file (Supports more iOS versions and devices, but this format will be deprecated at some point).
    """
    print('Building model...')
    print(sg_model)
    print('Model child nodes:')
    print(next(sg_model.named_children()))

    # Set the model in evaluation mode.
    sg_model.eval()
    print('Creating torch jit trace...')
    traced_model = torch.jit.trace(sg_model, example_inputs)
    print('Tracing the model with the provided inputs...')
    out = traced_model(*example_inputs)
    print('Inferred output shapes:', [o.shape for o in out])

    print('Converting to CoreML...')
    if not output_path:
        output_path = os.sep.join([os.getcwd(), sg_model.__class__.__name__.lower()])
        output_path += 'mlpackage' if export_as_ml_program else '.mlmodel'

    if export_as_ml_program:
        coreml_model = ct.convert(
            traced_model,
            convert_to="mlprogram",
            inputs=[ct.ImageType(name=f"x_{i+1}",
                                 shape=_.shape) for i, _ in enumerate(example_inputs)]
        )
    else:
        coreml_model = ct.convert(
            traced_model,
            inputs=[ct.ImageType(name=f"x_{i+1}",
                                 shape=_.shape) for i, _ in enumerate(example_inputs)]
        )

    spec = coreml_model.get_spec()
    print(spec.description)

    # Changing the input names:
    #   In CoreML, the input name is compiled into classes (named keyword argument in predict).
    #   We want to re-use the same names among different models to make research easier.
    #   We normalize the inputs names to be x_1, x_2, etc.
    for i, _input in enumerate(spec.description.input):
        new_input_name = "x_" + str(i + 1)
        print(f'Renaming input {_input.name} to {new_input_name}')
        ct.utils.rename_feature(spec, _input.name, new_input_name)

    # Re-Initializing the model with the new spec
    coreml_model = ct.models.MLModel(spec, weights_dir=coreml_model.weights_dir)

    # Saving the model
    coreml_model.save(output_path)
    print('CoreML model successfully save to ', os.path.abspath(output_path))


## Convert to CoreML

After the model loaded successfully, we will define a target path to store the compiled model into.<br>
In this case, our file is named "deci-yolo-nas-s.mlmodel" in the current working directory ('/content' in google colab).

In [4]:
output_path = "deci-yolo-nas-s.mlmodel"
convert_sg_model_to_coreml(sg_model=sg_torch_model,
                           output_path=output_path,
                           example_inputs=example_inputs,
                           export_as_ml_program=False)

Loaded successfully
Building model...
YoloNAS_S(
  (backbone): NStageBackbone(
    (stem): YoloNASStem(
      (conv): QARepVGGBlock(
        (nonlinearity): ReLU(inplace=True)
        (se): Identity()
        (post_bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (rbr_reparam): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      )
    )
    (stage1): YoloNASStage(
      (downsample): QARepVGGBlock(
        (nonlinearity): ReLU(inplace=True)
        (se): Identity()
        (post_bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (rbr_reparam): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      )
      (blocks): YoloNASCSPLayer(
        (conv1): Conv(
          (conv): Conv2d(96, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): ReLU(inplace=True)


[2023-05-15 10:57:17] WARNING - torchir_passes.py - Tuple detected at graph output. This will be flattened in the converted model.
[2023-05-15 10:57:17] INFO - converter.py - Converting graph.
[2023-05-15 10:57:17] INFO - builder.py - Adding op 'backbone.stem.conv.rbr_reparam.bias' of type const
[2023-05-15 10:57:17] INFO - builder.py - Adding op 'backbone.stem.conv.rbr_reparam.weight' of type const
[2023-05-15 10:57:17] INFO - builder.py - Adding op 'backbone.stem.conv.post_bn.running_var' of type const
[2023-05-15 10:57:17] INFO - builder.py - Adding op 'backbone.stem.conv.post_bn.running_mean' of type const
[2023-05-15 10:57:17] INFO - builder.py - Adding op 'backbone.stem.conv.post_bn.bias' of type const
[2023-05-15 10:57:17] INFO - builder.py - Adding op 'backbone.stem.conv.post_bn.weight' of type const
[2023-05-15 10:57:17] INFO - builder.py - Adding op 'backbone.stage1.downsample.rbr_reparam.bias' of type const
[2023-05-15 10:57:17] INFO - builder.py - Adding op 'backbone.stage1

input {
  name: "x_1"
  type {
    imageType {
      width: 640
      height: 640
      colorSpace: RGB
    }
  }
}
output {
  name: "var_1886"
  type {
    multiArrayType {
      dataType: FLOAT32
    }
  }
}
output {
  name: "var_1877"
  type {
    multiArrayType {
      dataType: FLOAT32
    }
  }
}
metadata {
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "torch==1.13.1+cu117"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "6.3.0"
  }
}

Renaming input x_1 to x_1


Let's look at our CoreML model:

In [6]:
ls -alh /content/

total 47M
drwxr-xr-x 1 root root 4.0K May 15 10:57 ./
drwxr-xr-x 1 root root 4.0K May 15 10:04 ../
drwxr-xr-x 4 root root 4.0K May 11 16:34 .config/
-rw-r--r-- 1 root root  47M May 15 10:57 deci-yolo-nas-s.mlmodel
drwxr-xr-x 2 root root 4.0K May 15 10:25 .ipynb_checkpoints/
drwxr-xr-x 1 root root 4.0K May 11 16:35 sample_data/


Cool! Our YOLO-NAS-S model weights 47MB in Float32 (full) percision.<br> It can definitely fit into an iOS app or OS X Desktop application.

## Visualize the CoreML model with Netron

If the notebook runs on localhost, you can visualize the exported CoreML model to Netron.<br>
If you use Google Colab, you can download the model in the directory explorer on the left, and upload it to https://netron.app/ instead of self-installing. 

In [ ]:
!python3 -m pip install netron

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import IPython
import threading
import time
import os

def display_netron(path):
    os.system(f'netron {path}')
    
thread = threading.Thread(target=display_netron, args=(output_path,))
thread.start()

time.sleep(1)
display(IPython.display.IFrame(f"http://localhost:8080", width=1000, height=1000))